### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [41]:
#b)
baby_df["review"] = baby_df["review"].fillna("")
#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [42]:
#a)
baby_df["review"] = baby_df["review"].apply(remove_punctuation)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock.,'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [43]:
#c)
baby_df = baby_df[baby_df["rating"] != 3]
#short test:
sum(baby_df["rating"] == 3)

0

In [44]:
#d) 
baby_df["rating"] = baby_df["rating"].apply(lambda x: 1 if x >= 4 else -1)
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [45]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names_out())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [46]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [47]:
#a)
train_df, test_df = train_test_split(baby_df, test_size=0.2, random_state=42)

In [48]:
#b)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_df["review"])
print(vectorizer.get_feature_names_out())

['00' '000' '0001' ... 'zzzzzz' 'zzzzzzz' 'zzzzzzzzzzz']


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [49]:
#a)
model = LogisticRegression()
model.max_iter = 1000
model.fit(X_train, train_df["rating"])


LogisticRegression(max_iter=1000)

In [50]:
#b)
print(model.coef_)
coefficients = model.coef_[0]
words = vectorizer.get_feature_names_out()
sorted_names = np.argsort(coefficients)
print("10 most negative words:")
for i in range(10):
    print(words[sorted_names[i]])
print("10 most positive words:")
for i in range(10):
    print(words[sorted_names[-i]])
#hint: model.coef_, vectorizer.get_feature_names()

[[ 0.00068141  0.01039348  0.00913426 ...  0.00820586  0.00390032
  -0.00015445]]
10 most negative words:
worthless
dissapointed
worst
useless
poorly
disappointing
unusable
disappointed
poor
unacceptable
10 most positive words:
worthless
lifesaver
minor
ply
con
thankful
rich
skeptical
saves
hinder


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [51]:
#a)
y_pred = model.predict(vectorizer.transform(test_df["review"]))
print(y_pred)

[ 1 -1 -1 ...  1  1  1]


In [52]:
#b)
y_pred_proba = model.predict_proba(vectorizer.transform(test_df["review"]))
print(y_pred_proba)
#hint: model.predict_proba()

[[4.53946852e-01 5.46053148e-01]
 [7.70482603e-01 2.29517397e-01]
 [9.69213808e-01 3.07861922e-02]
 ...
 [1.99302524e-05 9.99980070e-01]
 [1.07340294e-04 9.99892660e-01]
 [7.08034877e-02 9.29196512e-01]]


In [53]:
#c) 
best_reviews = np.argsort(y_pred_proba[:,1])[-5:]
worst_reviews = np.argsort(y_pred_proba[:,1])[:5]
print("5 most positive reviews:")
for i in range(5):
    print(test_df["review"].iloc[best_reviews[i]], "\n")

print("5 most negative reviews:")
for i in range(5):
    print(test_df["review"].iloc[worst_reviews[i]], "\n")
#hint: use the results of b)

5 most positive reviews:
I first bought these when I again had to replace the 12 pacifiers that had gone missing My daughter had grown accustom to the feel of the hospital brand pacifiers since she spent a couple of months there after birth I came across these and bought them because they were super cute but was unsure if my daughter would like them Previously I had bought pacifiers that looked like the ones the hospital provided but were not The plastic used to make them were shiny and for whatever reason my daughter had no interest in them I later discovered that the Gum Drop pacifiers were identical to the ones provided by hospitals They have a vast variety of styles and colors likeGumDrop Pacifier FullTerm Natural Scent Orange 5 Packand Gum Drop Preemie Pacifier Vanilla Scent Green for preemies These wubbanubs were in fact made of the same material and perfect for my little one I only wished I had found them sooner since my other two children are grown I could have saved myself fro

In [54]:
#d) 
accuracy = accuracy_score(test_df["rating"], y_pred)
print(accuracy)

0.9331054541093221


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [55]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [56]:
#a)
vectorizer = CountVectorizer(vocabulary=significant_words)
X_train = vectorizer.fit_transform(train_df["review"])
print(vectorizer.get_feature_names_out())

model_smaller = LogisticRegression()
model_smaller.max_iter = 1000
model_smaller.fit(X_train, train_df["rating"])
print(model_smaller.coef_)
coefficients = model_smaller.coef_[0]
words = vectorizer.get_feature_names_out()
sorted_names = np.argsort(coefficients)
print("10 most negative words:")
for i in range(10):
    print(words[sorted_names[i]])
print("10 most positive words:")
for i in range(10):
    print(words[sorted_names[-i]])

y_pred = model_smaller.predict(vectorizer.transform(test_df["review"]))
print(y_pred)

y_pred_proba = model_smaller.predict_proba(vectorizer.transform(test_df["review"]))
print(y_pred_proba)

best_reviews = np.argsort(y_pred_proba[:,1])[-5:]
worst_reviews = np.argsort(y_pred_proba[:,1])[:5]
print("5 most positive reviews:")
for i in range(5):
    print(test_df["review"].iloc[best_reviews[i]], "\n")

print("5 most negative reviews:")
for i in range(5):
    print(test_df["review"].iloc[worst_reviews[i]], "\n")
accuracy = accuracy_score(test_df["rating"], y_pred)
print(accuracy)

['love' 'great' 'easy' 'old' 'little' 'perfect' 'loves' 'well' 'able'
 'car' 'broke' 'less' 'even' 'waste' 'disappointed' 'work' 'product'
 'money' 'would' 'return']
[[ 1.35900018  0.93088153  1.19322423  0.07344137  0.50243139  1.51506778
   1.68497171  0.49619639  0.19326954  0.0745294  -1.68064024 -0.20157026
  -0.4897191  -1.97957053 -2.39875109 -0.63564887 -0.31372724 -0.94642423
  -0.34223904 -2.09283648]]
10 most negative words:
disappointed
return
waste
broke
money
work
even
would
product
less
10 most positive words:
disappointed
loves
perfect
love
easy
great
little
well
able
car
[1 1 1 ... 1 1 1]
[[0.07624749 0.92375251]
 [0.21395717 0.78604283]
 [0.21395717 0.78604283]
 ...
 [0.04229028 0.95770972]
 [0.10860443 0.89139557]
 [0.09165205 0.90834795]]
5 most positive reviews:
UPDATE 112013  I went ahead and used a tiny bit of WD40 and also thoroughly cleaned the carseat base and stroller and all the mechanisms started working like new again  No more issues opening or closing the

In [60]:
#b)
specific_word = "good"  # przykład konkretnego słowa
index = vectorizer.vocabulary_.get(specific_word)  # Indeks słowa w macierzy wektorów
print(f"Współczynnik dla słowa '{specific_word}': {coefficients[index]}")


Współczynnik dla słowa 'good': [[ 1.35900018  0.93088153  1.19322423  0.07344137  0.50243139  1.51506778
   1.68497171  0.49619639  0.19326954  0.0745294  -1.68064024 -0.20157026
  -0.4897191  -1.97957053 -2.39875109 -0.63564887 -0.31372724 -0.94642423
  -0.34223904 -2.09283648]]


In [58]:
#c)

time = %timeit -o model.predict(vectorizer.transform(test_df["review"]))
print(time)
time = %timeit -o model_smaller.predict(vectorizer.transform(test_df["review"]))
print(time)
#hint: %time, %timeit

2.28 s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.28 s ± 200 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


ValueError: X has 1 features, but LogisticRegression is expecting 20 features as input.